In [1]:
import pandas as pd
import numpy as np
import nltk
import os
import nltk.corpus

# Fonctions

In [12]:
def recup_words(text):
    fr = set(stopwords.words('french'))
    token = word_tokenize(text.lower())
    goodwords= [x for x in token if x not in fr]
    eng = set(stopwords.words('english'))
    goodwords2= [x for x in goodwords if x not in eng]
    return goodwords2

def list_freq(liste_words):
    allWords = []
    for words in liste_words:
        for w in words:
            if len(w)>1:
                allWords.append(w)
    freq=FreqDist(allWords)
    return freq

def text_mining_data_normalized(df,param,pays):
    categ_id = [24,23,22]
    
    df_top=pd.DataFrame()
    for i in categ_id:
        df_categ=df.loc[df['category_id']==i]
        df_categ=df_categ.sort_values(by = param,ascending=False)
        
        df_top=df_categ[:1000]
        
        liste_words=df_top['title'].apply(lambda x: recup_words(x))
        liste_words=liste_words.tolist()
    
    
        freq=list_freq(liste_words)
        freq_top = freq.most_common (40) 
        df_freq=pd.DataFrame(freq_top)
        df_freq.to_csv("Text_Mining/TextMining_"+pays+"_"+param+"_"+str(i)+".csv")

In [10]:
def moy_words(x,df,word_dict,param):
    
    df_unique=df.loc[df['title']==x]
    df_unique=df_unique.sort_values(by = param,ascending=False)
    df_unique=df_unique.iloc[0]
    fr = set(stopwords.words('french'))
    text=df_unique['title']
    views=df_unique[param]
    
    token = word_tokenize(text.lower())
    goodwords= [x for x in token if x not in fr]
    eng = set(stopwords.words('english'))
    goodwords2= [x for x in goodwords if x not in eng]
    goodwords2= [x for x in goodwords2 if len(x)>2]
    
    for word in goodwords2:
        
        if word in word_dict:
            word_dict["nb_"+word]+=1
            word_dict[word]=(word_dict[word]+views)/word_dict["nb_"+word]
            
            
        else:
            word_dict[word]=views
            word_dict["nb_"+word]=1
    return word_dict


def text_mining_moy(df,param,pays,categ):
    
    df=df.loc[df["category_id"]==categ]

    word_dict=dict()
    liste_words_views=df['title'].apply(lambda x: moy_words(x,df,word_dict,param))
    liste=liste_words_views.iloc[0]
    liste2=dict()
    for key in liste:
        if liste[key]>100:
            liste2[key]=liste[key]
            
    sortedDictDesc = sorted(liste2.items(),key=operator.itemgetter(1),reverse=True)
    
    df_moy=pd.DataFrame(sortedDictDesc)
    df_moy.to_csv("Text_Mining/TextMining_Moy_Best"+pays+"_"+param+"_"+str(categ)+".csv")

    sortedDictAsc = sorted(liste2.items(),key=operator.itemgetter(1),reverse=False)
    
    df_moy=pd.DataFrame(sortedDictAsc)
    df_moy.to_csv("Text_Mining/TextMining_Moy_Bad"+pays+"_"+param+"_"+str(categ)+".csv")
    

## Text mining totalement automatisé

In [13]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import operator

liste_csv=["FR","GB","CA","US"]
for pays in liste_csv:
    df=pd.read_csv(pays+"videos.csv")
    
    # Dataframe des 100 vidéos ayant fait le plus de vues par catégories
    param="views"
    text_mining_data_normalized(df,param,pays)
        
    # Dataframe des 100 vidéos ayant fait le plus de likes par catégories
    param="likes"
    text_mining_data_normalized(df,param,pays)
    
    # Rappel Catégories: id 24= Entertainement  , 23= Comedy , 22= people

    
    # Dataframe des moyennes par likes
    param="likes"
    text_mining_moy(df,param,pays,22)
    text_mining_moy(df,param,pays,23)
    text_mining_moy(df,param,pays,24)
    
    # Dataframe des moyennes par vues
    
    param="views"
    text_mining_moy(df,param,pays,22)
    text_mining_moy(df,param,pays,23)
    text_mining_moy(df,param,pays,24)


## Téléchargements à faire si besoin

In [5]:
## Téléchargement de la librairie nlk permettant le text mining
nltk.download()
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /Users/dylamano/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True